In [14]:
import tkinter as tk
from tkinter import ttk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import os

def carregar_dados():
    palavras = []
    rotulos = []

    caminho = "palavras.txt"

    if os.path.exists(caminho):
        with open(caminho, 'r') as f:
            for linha in f:
                try:
                    palavra,rotulo = linha.strip().split(',')
                    palavras.append(palavra)
                    rotulos.append(int(rotulo))

                except ValueError:
                    print(f"Erro ao processar a linha: {linha.strip()}")
        
    return palavras, rotulos

def salvar_dados(palavras,rotulos):
    caminho = "palavras.txt"
    with open(caminho, 'w') as f:
        for palavra,rotulo in zip(palavras,rotulos):
            f.write(f"{palavra},{rotulo}\n")

def treinar_modelo(palavras,rotulos):
    modelo = make_pipeline(CountVectorizer(),MultinomialNB())#countvectorizer(transforma as palavras em vetores numericos)/MultinominalNB(aplica o algoritmo de clasivicaçãp naive bayes)

    modelo.fit(palavras,rotulos)

    return modelo

palavras,rotulos = carregar_dados()

def adicionar_palavra():
    global modelo,palavras,rotulos

    nova_palavra = entrada_palavra.get()
    novo_rotulo = var_rotulo.get()

    if nova_palavra and (novo_rotulo == 0 or novo_rotulo == 1):
        if nova_palavra not in palavras:
            palavras.append(nova_palavra.lower())
            rotulos.append(novo_rotulo)

            salvar_dados(palavras,rotulos)

            modelo = treinar_modelo(palavras,rotulos)

            lbl_resultado['text'] = f"A palavra '{nova_palavra}' foi adicionada e o modelo foi atualizado."
        
        else:
            lbl_resultado['text'] = f"A palavra '{nova_palavra}' já foi cadastrada"

def verificar_palavra():
    global modelo,palavras,rotulos

    palavras,rotulos = carregar_dados()
    modelo = treinar_modelo(palavras,rotulos)

    consultar_palavra = entrada_palavra.get().lower()

    if consultar_palavra in palavras:
        previsao = modelo.predict([consultar_palavra])
        lbl_resultado['text'] = f"Previsão para '{consultar_palavra}':{'é uma fruta' if previsao[0]==1 else 'não é uma fruta'}"
    else:
        lbl_resultado['text'] = f"A palavra '{consultar_palavra}' não encontrada. Por favor, adicione-o primeiro"


janela = tk.Tk()
janela.geometry("900x300")
janela.title("Classificador de frutas")

fonte_titulo = ("Arial",24)
fonte_texto = ("Arial",16)
cor_fundo = "#f2f2f2"
cor_botao = "#4caf50"
cor_texto_botao = "#ffffff"

janela.configure(bg=cor_fundo)

lbl_instrucao = tk.Label(janela,
                         text="Insira uma palavra: ",
                         font=fonte_titulo,bg=cor_fundo)
lbl_instrucao.grid(row=0,column=0,padx=20,pady=20)

entrada_palavra = tk.Entry(janela,font=fonte_texto)
entrada_palavra.grid(row=0,column=1,padx=20,pady=20)

var_rotulo = tk.IntVar()
var_rotulo.set(0)

r1 = tk.Radiobutton(janela,
                    text="Não é uma fruta",
                    variable=var_rotulo,
                    value=0,
                    font=fonte_texto,
                    bg=cor_fundo)
r1.grid(row=1,column=0,padx=20,pady=10)

r2 = tk.Radiobutton(janela,
                    text="É uma fruta",
                    variable=var_rotulo,
                    value=1,
                    font=fonte_texto,
                    bg=cor_fundo)
r2.grid(row=1,column=1,padx=20,pady=10)

btn_adicionar = tk.Button(janela,
                          text="Adicionar Palavra",
                          command=adicionar_palavra,
                          font=fonte_texto,
                          bg=cor_botao,
                          fg=cor_texto_botao)
btn_adicionar.grid(row=2,column=0,padx=20,pady=20)

btn_verificar = tk.Button(janela,
                          text="Verificar Palavra",
                          command=verificar_palavra,
                          font=fonte_texto,
                          bg=cor_botao,
                          fg=cor_texto_botao)
btn_verificar.grid(row=2,column=1,padx=20,pady=20)

lbl_resultado = tk.Label(janela,
                         text="Resultado",
                         font=("Arial",20),
                         bg=cor_fundo)
lbl_resultado.grid(row=3,columnspan=2,padx=20,pady=20)

janela.mainloop()


In [19]:
import tkinter as tk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import numpy as np

perguntas_treino = ["Qual é seu nome?", "Como você está?","O que você faz?","Você gosta de programar?"]
respostas_treino = ["Meu nome é Bruno Pereira", "Estou mal", "Eu vivo","Adoro!"]

vectorizer = TfidfVectorizer()
modelo = make_pipeline(vectorizer,MultinomialNB())
modelo.fit(perguntas_treino,np.arange(len(respostas_treino)))

def responder_pergunta():
    pergunta = entrada_pergunta.get()

    if pergunta:
        idx = modelo.predict([pergunta])[0]

        prob = np.max(modelo.predict_proba([pergunta])) 

        if prob > 0.3:
            lbl_resposta["text"] = f"Resposta: {respostas_treino[idx]}"
        else:
            lbl_resposta["text"] = f"Resposta não encontrada"

janela = tk.Tk()
janela.geometry("600x300")
janela.title("Sistema de Resposta Automática")

lbl_instrucao = tk.Label(janela,
                         text="Faça uma pergunta: ",font=("Arial",16))
lbl_instrucao.pack(pady=10)
entrada_pergunta = tk.Entry(janela,width=50,font=("Arial",16))
entrada_pergunta.pack(pady=5)
btn_responder = tk.Button(janela,
                          text="Responder",
                          command=responder_pergunta,font=("Arial",16))
btn_responder.pack(pady=10)

lbl_resposta = tk.Label(janela,
                        text="",
                        font=("Arial",16))
lbl_resposta.pack(pady=10)


janela.mainloop()


In [7]:
###pip install autocorrect
###pip install python-Levenshtein
import tkinter as tk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
import numpy as np
import os
from Levenshtein import distance as levenshtein_distance

def carregar_dados():
    palavras = []
    respostas = []

    caminho = "perguntas.txt"

    if os.path.exists(caminho):
        with open(caminho, 'r') as f:
            for linha in f:
                try:
                    palavra,resposta = linha.strip().split('|')
                    palavras.append(palavra)
                    respostas.append(resposta)

                except ValueError:
                    print(f"Erro ao processar a linha: {linha.strip()}")
        
    return palavras, respostas    

def corrigir_texto(texto,vocabulario):
    palavras_corrigidas = []

    for palavra in texto.split():
        min_dist = float("inf")
        palavra_correta = palavra

        for vocab in vocabulario:
            dist = levenshtein_distance(palavra,vocab)
            if dist < min_dist:
                min_dist = dist
                palavra_correta = vocab
                
        palavras_corrigidas.append(palavra_correta)

    return " ".join(palavras_corrigidas)

perguntas_treino, respostas_treino = carregar_dados()
vocabulario = set(" ".join(perguntas_treino).split())

def responder_pergunta():
    if not perguntas_treino or not respostas_treino:
        caixa_resposta.delete(1.0,tk.END)
        caixa_resposta.insert(tk.END,"O conjunto de treinamento está vazio")

    pergunta = entrada_pergunta.get()
    pergunta_corrigida = corrigir_texto(pergunta,vocabulario)

    print(pergunta_corrigida)

    if pergunta_corrigida:
        idx = modelo.predict([pergunta_corrigida])[0]
        prob = np.max(modelo.predict_proba([pergunta_corrigida]))

        print(f"Probabilidade: {prob}")

        caixa_resposta.delete(1.0,tk.END)
        caixa_resposta.insert(tk.END,f"Resposta: {respostas_treino[idx]}")

if perguntas_treino and respostas_treino:
    vectorizer = TfidfVectorizer()
    modelo = make_pipeline(vectorizer,MultinomialNB())
    modelo.fit(perguntas_treino,np.arange(len(respostas_treino)))

janela = tk.Tk()
janela.geometry("700x350")
janela.title("Sistema de Resposta Automática")

lbl_instrucao = tk.Label(janela,text="Faça uma pergunta: ",font=("Arial",16))
lbl_instrucao.pack(pady=10)
entrada_pergunta = tk.Entry(janela,width=50,font=("Arial",16))
entrada_pergunta.pack(pady=5)
btn_responder = tk.Button(janela,text="Responder",command=responder_pergunta,font=("Arial",16))
btn_responder.pack(pady=10)

caixa_resposta = tk.Text(janela,wrap=tk.WORD,width=50,height=10,font=("Arial",16))
caixa_resposta.pack(pady=10)

janela.mainloop()

In [16]:
import tkinter as tk
from Levenshtein import distance as levenshtein_distance
import re

respostas = {
    'bom dia': 'Bom dia! Como posso ajudá-lo?',
    'boa tarde': 'Boa tarde! Como posso ajudá-lo?',
    'boa noite': 'Boa noite! Como posso ajudá-lo?',
    'olá': 'Olá! Como posso ajudá-lo?',
    'oi': 'Oi! Como posso ajudá-lo?',
    'como você está': 'Estou funcionando conforme programado. E você?',
    'qual seu nome': 'Eu sou um chatbot e não tenho um nome.',
    'o que você faz': 'Eu estou aqui para responder suas perguntas e ajudá-lo.',
    'você é um robô': 'Sim, eu sou um programa de computador projetado para conversar com você.',
    'adeus': 'Até logo! Se precisar, estarei aqui.',
    'tchau': 'Tchau! Foi bom conversar com você.',
    'quem te criou': 'Fui criado por um programador.',
    'você gosta de música': 'Não tenho preferências, sou apenas um programa.',
    'você pode me ajudar': 'Claro, em que posso ajudá-lo?',
    'como funciona': 'Você faz perguntas e eu tento responder da melhor forma possível.',
    'você é inteligente': 'Eu tento ser o mais útil possível.',
    'você tem sentimentos': 'Não, eu sou apenas um conjunto de algoritmos.',
    'qual a sua idade': 'A idade não se aplica a mim.',
    'você está vivo': 'Não, eu sou apenas um programa de computador.',
    'como está o tempo': 'Não posso acessar a internet, então não sei como está o tempo.',
    'você gosta de café': 'Não tenho preferências ou gostos.',
    'qual é o seu filme favorito': 'Não assisto filmes, sou apenas um programa.',
    'você pode aprender': 'Eu não tenho a capacidade de aprender, sou programado para responder com base em um conjunto predefinido de respostas.',
    'você é feliz': 'Não tenho emoções.',
    'você é casado': 'Não, eu sou apenas um programa de computador.',
    'você tem filhos': 'Não, eu não tenho a capacidade de ter filhos.',
    'você tem amigos': 'Não, eu sou apenas um programa isolado.',
    'onde você mora': 'Eu existo em um servidor.',
    'você pode me contar uma piada': 'Por que os programadores preferem o escuro? Porque a luz atrai bugs.',
    'você é humano': 'Não, eu sou uma máquina.',
    'qual é o seu esporte favorito': 'Não tenho um esporte favorito, sou apenas um programa.',
    'você gosta de viajar': 'Não posso viajar, estou confinado a um servidor.',
    'você pode dirigir': 'Não, eu não tenho essa capacidade.',
    'você come': 'Não, eu não preciso de comida.',
    'você dorme': 'Não, eu estou sempre ativo.',
    'você pode cantar': 'Não, eu não tenho essa funcionalidade.',
    'você tem um animal de estimação': 'Não, eu não posso cuidar de animais de estimação.',
    'você gosta de ler': 'Não posso ler no sentido humano, mas posso processar texto.',
    'você pode dançar': 'Não, eu não tenho forma física para dançar.',
    'qual é o seu trabalho': 'Estou programado para conversar e fornecer respostas com base no meu conjunto de dados.',
    'você gosta de música': 'Não tenho a capacidade de gostar ou desgostar de música.',
    'você pode ver': 'Não, eu não tenho sensores visuais.',
    'qual é o seu nome': 'Eu sou um chatbot sem um nome específico.',
    'você gosta de matemática': 'Eu sou programado para processar números, mas não tenho preferências.',
    'você pode ler minha mente': 'Não, eu não tenho essa capacidade.',
    'você é inteligente': 'Minha inteligência é limitada ao que fui programado para fazer.',
    'você pode me ajudar com a lição de casa': 'Depende da pergunta, posso tentar ajudar.',
    'você pode fazer café': 'Não, eu não tenho capacidades físicas.',
    'você pode abrir a porta': 'Não, eu não posso interagir fisicamente com o ambiente.',
    'você tem um celular': 'Não, eu não tenho dispositivos físicos.',
    'como você foi criado': 'Fui criado usando algoritmos de programação.',
    'você pode ficar doente': 'Não, eu não tenho uma forma biológica.',
    'você pode ir à escola': 'Não, mas posso fornecer informações que são ensinadas nas escolas.',
    'qual é o seu hobby': 'Eu não tenho hobbies, estou aqui para responder perguntas.',
    'você tem um carro': 'Não, eu não tenho bens físicos.',
    'você pode voar': 'Não, eu não tenho capacidades físicas.',
    'você pode nadar': 'Não, eu não interajo com o ambiente físico.',
    'você pode correr': 'Não, eu não tenho forma física.',
    'você pode pular': 'Não, eu não tenho a capacidade de movimento.'
}

def encontra_melhor_resposta(mensagem,respostas):
    melhor_resposta = "Desculpa, não entendi essa pergunta"
    melhor_pontuação = 0

    for pergunta,resposta in respostas.items():
        palavras_pergunta = set(re.findall(r'\b\w+\b',pergunta.lower()))
        palavras_mensagem = set(re.findall(r'\b\w+\b',mensagem.lower()))

        pontuacao = len(palavras_pergunta.intersection(palavras_mensagem))

        if pontuacao > melhor_pontuação:
            melhor_pontuação = pontuacao
            melhor_resposta = resposta
    
    return melhor_resposta


def enviar_mensagem():
    mensagem = entrada.get()

    if not mensagem:
        return
    
    canvas.create_text(790,canvas.y,text=mensagem,font=("Arial",20),anchor="e",tags='texto',fill='blue')
    canvas.y += 30

    resposta = encontra_melhor_resposta(mensagem,respostas)
    canvas.create_text(20, canvas.y, font=("Arial 20"), text=resposta, anchor='w', tags='texto', fill='green')
    canvas.y += 30

    entrada.delete(0,tk.END)
    

janela = tk.Tk()
janela.title("Chatbot")

canvas = tk.Canvas(janela,bg="white",width=500,height=400)
canvas.pack(expand=tk.YES,fill=tk.BOTH)
canvas.y = 20

entrada = tk.Entry(janela,font=("Arial",20),width=50)
entrada.pack(side=tk.LEFT,padx=10,pady=10)

btn_enviar = tk.Button(janela,text="Enviar",command=enviar_mensagem,font=("Arial",20))
btn_enviar.pack(side=tk.RIGHT,padx=10,pady=10)

janela.mainloop()

In [8]:
import tkinter as tk
from tkinter import ttk
from tkinter import font
from tkinter import Text
import re
import requests

def obter_cotacao_dolar():
    api_url = "https://api.exchangerate-api.com/v4/latest/USD"
    
    try:
        resposta = requests.get(api_url, verify=False)
        dados = resposta.json()
        return dados['rates']['BRL']
    except Exception as e:
        return f"Erro ao obter cotação: {e}"

def somar_numeros(numeros):
    return sum(numeros)

def subtrair_numeros(numeros):
    return numeros[0] - numeros[1]

def multiplicar_numeros(numeros):
    resultado = 1

    for num in numeros:
        resultado *= num
    return resultado

def dividir_numeros(numeros):
    
    try:
        resultado = numeros[0]
        for num in numeros[1:]:
            resultado /= num
        return resultado
    except ZeroDivisionError:
        return "Divisão por zero não permitida"
    
def tratar_mensagem(mensagem):
    mensagem = mensagem.lower()
    numeros = [int(x) for x in re.findall(r'\d+', mensagem)]

    respostas = []
    if "some" in mensagem:
        if numeros:
            respostas.append(f"A soma dos números é {somar_numeros(numeros)}")

    if "subtraia" in mensagem:
        if numeros:
            respostas.append(f"A subtração dos números é {subtrair_numeros(numeros)}")
    if "multiplique" in mensagem:
        if numeros:
            respostas.append(f"A multiplicação dos números é {multiplicar_numeros(numeros)}")
    if "divida" in mensagem:
        if numeros:
            respostas.append(f"A divisão dos números é {dividir_numeros(numeros)}")
    if "dólar" in mensagem:
        cotacao = obter_cotacao_dolar()
        respostas.append(f"A cotação atual do dólar é {cotacao} BRL")

    palavras = mensagem.split()

    palavras_tratadas = []

    for palavra in palavras:
        if re.fullmatch(r'\d+', palavra):
            palavras_tratadas.append(f"[{palavra}]")
        elif palavra in ["some", "subtraia", "multiplique", "divida"]:
            palavras_tratadas.append(palavra.upper())
        else:
            palavras_tratadas.append(palavra)

    mensagem_tratada = " ".join(palavras_tratadas)

    text_area.config(state=tk.NORMAL)
    pos_fim_usuario = text_area.index(tk.INSERT)
    pos_inicio_usuario = text_area.index(tk.INSERT)

    if respostas:
        resposta = "\n".join(respostas)
        text_area.insert(tk.END, f"Bot: {resposta}\n\n")
        pos_fim_bot = text_area.index(tk.INSERT)
        text_area.tag_add("cor_bot", pos_fim_usuario, pos_fim_bot)
        text_area.tag_config("cor_bot", foreground="green")
    else:
        text_area.insert(tk.END, "Bot: Desculpe, não entendi sua solicitação.\n\n")

    text_area.config(state=tk.DISABLED)

    return "\n".join(respostas) if respostas else "Desculpe, não entendi sua solicitação."

def enviar_mensagem():
    
    mensagem = entrada.get()
    
    if not mensagem:
        return
    
    text_area.config(state=tk.NORMAL)
    text_area.insert(tk.END, f"Você: {mensagem}\n")
    
    resposta = tratar_mensagem(mensagem)
    inicio = text_area.index(tk.INSERT)

    text_area.insert(tk.END, f"Bot: {resposta}\n")
    
    fim = text_area.index(tk.INSERT)
    
    text_area.tag_add("resposta_bot", inicio, fim)
    text_area.tag_config("resposta_bot", foreground="green")
    text_area.config(state=tk.DISABLED)

    entrada.delete(0, tk.END)
    text_area.see(tk.END)
    
    
janela = tk.Tk()
janela.title('Chatbot')

text_area = Text(janela, font=("Arial 14"), wrap=tk.WORD, width=50, height=20)
text_area.pack(expand=tk.YES, fill=tk.BOTH)
text_area.config(state=tk.DISABLED)

entrada = tk.Entry(janela, font=("Arial", 20), width=50)
entrada.pack(side=tk.LEFT, padx=10, pady=10)

btn_enviar = tk.Button(janela, text='Enviar', command=enviar_mensagem)
btn_enviar.pack(side=tk.RIGHT, padx=10, pady=10)

janela.mainloop()

c:\Users\bruno.pereira.silva\AppData\Local\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.exchangerate-api.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
